In [12]:
import pandas as pd
import glob
import numpy as np
import torch
import math

In [13]:
data = glob.glob('Data/*.npz')
labels = glob.glob('Data/*.csv')
data.sort()
labels.sort()

In [14]:
patch_size = 32
num_patches = 100
stride = 32
X = []
y = []

In [15]:
for i in range(len(labels)):
    arr = np.load(data[i])['arr_0']
    
    arr = torch.tensor(arr[-1])
    lab = torch.zeros(len(arr))
    arr = arr.unfold(0, patch_size, stride).unfold(0, num_patches, 1).movedim(-1, 1)
    X.append(arr)

    df = pd.read_csv(labels[i], header=None)
    for i in range(len(df)):
        lab[df.iloc[i,0]:df.iloc[i,1]] = df.iloc[i,2]
    lab = lab.unfold(0, patch_size, stride).unfold(0, num_patches, 1).movedim(-1, 1)
    anom_count = torch.count_nonzero(lab, dim=-1).float() / patch_size
    norm_count = 1. - anom_count
    lab_count = torch.stack((norm_count, anom_count), dim=-1)


    y.append(anom_count)
    # print(lab_count[9937, 0])
    print(arr.shape, lab_count.shape)

torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size([9373, 100, 32]) torch.Size([9373, 100, 2])
torch.Size

In [16]:
X = torch.cat(X, dim=0)
y = torch.cat(y, dim=0)

In [17]:
X.shape, y.shape

(torch.Size([431158, 100, 32]), torch.Size([431158, 100]))

In [18]:
torch.save(X, f'processed_data/X_soft_ps{patch_size}_np{num_patches}.pt')
torch.save(y, f'processed_data/y_soft_ps{patch_size}_np{num_patches}.pt')

In [19]:
X = []
for i in range(len(labels)):
    arr = np.load(data[i])['arr_0']
    X.append(torch.tensor(arr[-1]))
X = torch.cat(X)
X.shape

torch.Size([13942784])

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X.unsqueeze(-1))

In [21]:
scaler.mean_, scaler.var_

(array([3.9900535e-06]), array([9.19630804e-07]))

In [22]:
import joblib

joblib.dump(scaler, 'scalers/standard_scaler_4.gz')

['scalers/standard_scaler_4.gz']